<a href="https://colab.research.google.com/github/Bish-Soli/MQP/blob/main/MQP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

In [5]:
dataset = load_dataset('cifar100')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [11]:
from torch.utils.data import DataLoader
from torchvision import transforms

In [13]:
dataloader = DataLoader(dataset)

In [14]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:03<00:00, 48918190.39it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [49]:
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, weight_normalization = False):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.weight_normalization = weight_normalization


    def forward(self, x):
        if self.weight_normalization == True:
           self.max_norm_(self.conv.weight)
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x= self.pool(x)

        return x

    def max_norm_(self, w, _max_norm_val = 3):
      with torch.no_grad():
        norm = w.norm(2, dim=0, keepdim=True).clamp(min=_max_norm_val / 2)
        desired = torch.clamp(norm, max=_max_norm_val)
        w *= (desired / norm)



class Net(nn.Module):
    def __init__(self, weight_normalization = False):
        super().__init__()
        self.conv1 = ConvBlock(3, 6, 5, weight_normalization=weight_normalization)
        self.conv2 = ConvBlock(6, 16, 5, weight_normalization=weight_normalization)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x





In [50]:
class ClassSetup:

  def __init__(self,lr = 0.001, weight_decay=0, weight_normalization= False,opt='Adam', loss_type = 'metric'):
    self.weight_decay = weight_decay
    self.weight_normalization = weight_normalization
    self.lr = lr
    self.model = Net(weight_normalization)
    self.opt = self.get_optimizer(opt)
    self.loss = self.get_loss(loss_type)


  def get_optimizer(self, opt):
    if opt == 'Adam':
      return torch.optim.Adam(self.model.parameters(), lr = self.lr, weight_decay = self.weight_decay)
    else:
     return torch.optim.SGD(self.model.parameters(), lr = self.lr, weight_decay = self.weight_decay)

  def get_loss(self, loss_type):
    if loss_type =='metric':
      return nn.CrossEntropyLoss()
    # elif self.loss_type == 'contrastive':
    #   self.loss = ContrastiveLoss()
    # else
    #   self.loss = Hybrid()

  def get_all (self):
    return self.opt, self.loss, self.model




In [51]:
net = Net()
setup = ClassSetup(weight_normalization=True, weight_decay=1e-4)
optimizer, criterion, net = setup.get_all()
net.to(device)

Net(
  (conv1): ConvBlock(
    (conv): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (bn): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): ConvBlock(
    (conv): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [52]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.866
[1,  4000] loss: 1.665


KeyboardInterrupt: 

cpu
